In [1]:
import gevent

In [2]:
def foo():
    print('[1] Running in foo')
    gevent.sleep(0)
    print('[2] Emplict context switch to foo again')

def bar():
    print('[3] Emplict context to bar')
    gevent.sleep(0)
    print('[4] implicit switch switch back to bar')

gevent.joinall([
    gevent.spawn(foo),
    gevent.spawn(bar),
])

[1] Running in foo
[3] Emplict context to bar
[2] Emplict context switch to foo again
[4] implicit switch switch back to bar


[<Greenlet at 0x7ffab87543b0: _run>, <Greenlet at 0x7ffab87544d0: _run>]

## Synchronous & Asynchronous Execution

In [3]:
import time
from gevent import select

In [4]:
# gevent의 진짜 힘은 상호작용 하며 스케쥴링 될 수 있는 네트워크와 IO bound 함수들을 작성할 때 발휘된다.
# gevent는 네트워크 라이브러리들이 암시적으로 greenlet 컨텍스트들이 가능한 시점에 암시적으로 yield 하도록 보장합니다 (==> 뭔 소리인지...)

start = time.time()
tic = lambda: 'at %1.1f seconds' % (time.time() - start)

def gr1():
    # Busy waits for a second, but we don't want to stick around...
    print('[1] Started Polling: %s' % tic())
    select.select([], [], [], 2)
    print('[2] Ended Polling: %s' % tic())

def gr2():
    # Busy waits for a second, but we don't want to stick around...
    print('[3] Started Polling: %s' % tic())
    select.select([], [], [], 2)
    print('[4] Ended Polling: %s' % tic())

def gr3():
    print("[5] Hey lets do some stuff while the greenlets poll, %s" % tic())
    gevent.sleep(1)

gevent.joinall([
    gevent.spawn(gr1),
    gevent.spawn(gr2),
    gevent.spawn(gr3),
])

[1] Started Polling: at 0.0 seconds
[3] Started Polling: at 0.0 seconds
[5] Hey lets do some stuff while the greenlets poll, at 0.0 seconds
[2] Ended Polling: at 2.0 seconds
[4] Ended Polling: at 2.0 seconds


[<Greenlet at 0x7ffab8754710: _run>,
 <Greenlet at 0x7ffab8754290: _run>,
 <Greenlet at 0x7ffab87545f0: _run>]

> `.spawn(?)`에 들어가면 coroutine에 들어간다고 보면 될 거 같다..

In [5]:
import random

def task(pid):
    """
    Some non-deterministic task
    """
    gevent.sleep(random.randint(0, 2) * 0.001)
    print("Task %s done" % pid)

def synchronous():
    for i in range(1, 10):
        task(i)

def asynchronous():
    threads = [gevent.spawn(task, i) for i in range(10)]
    gevent.joinall(threads)

print('Synchronous:')
synchronous()

print('Asynchronous:')
asynchronous()

Synchronous:
Task 1 done
Task 2 done
Task 3 done
Task 4 done
Task 5 done
Task 6 done
Task 7 done
Task 8 done
Task 9 done
Asynchronous:
Task 1 done
Task 8 done
Task 9 done
Task 2 done
Task 5 done
Task 6 done
Task 0 done
Task 3 done
Task 4 done
Task 7 done


> 동기(synchronous)처리 시 모든 task들이 순차적으로 실행되고, 다른 task들이 각각 동작하는 동한 blocking 방식으로 동작한다. <br/>
비동기(asynchronous)처리 시 실행 순서가 보장되지 않고 실행시간이 동기처리 시 보다 훨씬 줄어든다. task들이 서로  실행을 block 하지 않기에

In [15]:
import gevent.monkey
gevent.monkey.patch_socket()

import urllib.request
import simplejson as json

def fetch(pid):
    response = urllib.request.urlopen('http://worldtimeapi.org/api/timezone/America/Argentina/Salta')
    result = response.read()
    json_result = json.loads(result)
    datetime = json_result['datetime']
    
    print('Process %s: %s' % (pid, datetime))
    return json_result['datetime']

def synchronous():
    for i in range(1, 10):
        fetch(i)

def asynchronous():
    threads = []
    for i in range(1, 10):
        threads.append(gevent.spawn(fetch, i))
    gevent.joinall(threads)

print('Synchronous:')
synchronous()

print('Asynchronous:')
asynchronous()

Synchronous:
Process 1: 2020-12-10T05:21:44.864725-03:00
Process 2: 2020-12-10T05:21:46.436948-03:00
Process 3: 2020-12-10T05:21:47.873143-03:00
Process 4: 2020-12-10T05:21:49.022463-03:00
Process 5: 2020-12-10T05:21:49.803659-03:00
Process 6: 2020-12-10T05:21:50.470372-03:00
Process 7: 2020-12-10T05:21:51.049627-03:00
Process 8: 2020-12-10T05:21:51.612511-03:00
Process 9: 2020-12-10T05:21:52.172603-03:00
Asynchronous:
Process 3: 2020-12-10T05:21:52.735115-03:00
Process 8: 2020-12-10T05:21:52.740205-03:00
Process 5: 2020-12-10T05:21:52.732950-03:00
Process 2: 2020-12-10T05:21:52.738408-03:00
Process 1: 2020-12-10T05:21:52.739684-03:00
Process 7: 2020-12-10T05:21:52.742091-03:00
Process 6: 2020-12-10T05:21:52.745032-03:00
Process 4: 2020-12-10T05:21:52.745415-03:00
Process 9: 2020-12-10T05:21:52.748487-03:00
